In [64]:
from google.cloud import bigquery
import pandas as pd

In [65]:

time = '2025-06-25'
SQL_QUERY =  f"""
SELECT
  sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type
FROM
  `htz-common.raw_data.requests` 
WHERE
  TIMESTAMP_TRUNC(event_time, DAY) >= TIMESTAMP('{time}')
  AND page_type = 'Chatbot'
  AND campaign_name in ('response answer','open question')
  -- AND sso_id IS NOT NULL
ORDER BY
  event_time DESC;
"""
client = bigquery.Client()
query_job = client.query(SQL_QUERY)

results = query_job.result() 
df = results.to_dataframe()
# add index
df['index'] = df.index
df['event_time_temp'] = df['event_time'].dt.strftime('%Y-%m-%d %H:%M')
df['event_time'] = df['event_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df = df.drop_duplicates(subset=['sso_id', 'campaign_details','event_time_temp'], keep='last')
df = df.drop(columns=['event_time_temp'])
print(df.shape[0])
df = df.reset_index(drop=True)
df.head(2)

/Users/roeishlezinger/Desktop/movie-recommendation-chatbot/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


56


,sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type,index
0,9037014398,"[{'key': 'total_time', 'value': '1.57545804977...",טרולים?אולי תתמכרו דווקא ל<strong>טרולים</stro...,response answer,App,2025-06-25 11:45:27,impression,0
1,9037014398,"[{'key': 'session_id', 'value': '3'}]",מהמהמה,open question,App,2025-06-25 11:45:25,action,1


In [66]:
# preprocess the data

df['parsed_data'] = df['additional_info'].apply(lambda x: {item['key']: item['value'] for item in x})
# add index to parsed data
expanded_df = pd.json_normalize(df['parsed_data'].tolist())

expanded_df = expanded_df.drop(columns=['sso_id'])
df_cleaned = df.drop(columns=['additional_info', 'parsed_data'])
df = pd.concat([df_cleaned, expanded_df], axis=1)
df['id'] = df['sso_id'].astype(str) + '_' + df['session_id'].astype(str)

df.head(2)

,sso_id,campaign_details,campaign_name,platform,event_time,record_type,index,total_time,rag_speed,session_id,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id
0,9037014398,טרולים?אולי תתמכרו דווקא ל<strong>טרולים</stro...,response answer,App,2025-06-25 11:45:27,impression,0,1.57545804977417,3.16619873046875E-4,3,...,0000017f-e071-d804-ad7f-f1fbb77c0000,0,199,true,9037014398_3,,false,,,9037014398_3
1,9037014398,מהמהמה,open question,App,2025-06-25 11:45:25,action,1,NaN,NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9037014398_3


In [67]:
df['id'].value_counts().sort_values(ascending=False).head(10)

id
8010962861_117    10
8010962861_121     9
9037014398_3       8
8010962861_118     7
9037014398_4       6
8010962861_124     5
8010962861_123     4
50000227213_12     4
8010962861_120     2
8010962861_119     1
Name: count, dtype: int64

In [70]:
def parse_conversation(ids:str,time:bool) -> str:
    """
    Parse the conversation from the DataFrame based on the provided id.
    """
    conversation = df[df['id'] == ids]
    conversation = conversation.sort_values(by='event_time', ascending=True)
    if conversation.empty:
        return "No conversation found for this ID."
    
    conversation_text = []
    for _, row in conversation.iterrows():
        if row['campaign_name'] == 'open question':
            if time:
                text = f"{row['event_time']} - USER: {row['campaign_details']}"
            else:
                text = f"USER: {row['campaign_details']}"
        elif row['campaign_name'] == 'response answer':
            if time:
                text = f"{row['event_time']} - LLM: {row['campaign_details']}"
            else:
                text = f"LLM: {row['campaign_details']}"
        conversation_text.append(text)
    
    return "\n".join(conversation_text)

txt = parse_conversation('8010962861_117', True)
print(txt)

2025-06-25 04:47:50 - USER: בוקר טוב
2025-06-25 04:48:17 - USER: סרט באמזון. שיהיה טוב, אני פתוח לדרמות, קומדיות ומצוייר. לא מתח
2025-06-25 04:48:21 - LLM: <strong>אורון שמיר</strong> ממליץ על הסרט <strong>"אחים"</strong> באמזון פריים וידיאו. הוא מתאר אותו כקומדיה חביבה עם ג'וש ברולין ופיטר דינקלג', שיכולה להיות צפייה ביתית נעימה. למרות הציפיות הגבוהות מהבמאי, הסרט מוגדר כחביב ותו לא. כמו כן, ישנה גם ביקורת של <strong>אורון שמיר</strong> על הסרט <strong>"מושבע מס' 2"</strong>, סרט פרישה של קלינט איסטווד, שזמין להשכרה בתשלום באמזון (בין היתר). הוא מתאר אותו כמותחן משפטי קלאסי וסולידי.
2025-06-25 04:48:56 - USER: אוקי, אז משהו ממש ממש טוב באמזון
2025-06-25 04:51:11 - LLM: אני מבין שאתם רוצים המלצה על משהו <strong>ממש ממש טוב</strong> באמזון. חשוב לי לציין שאין לי יכולת לדרג סרטים או לדעת מה ה"טוב ביותר" באופן אובייקטיבי, מכיוון שאין לי גישה לנתוני דירוגים או פופולריות. עם זאת, אשמח להמליץ לכם על סרטים שזכו לביקורות חיוביות על בסיס התוכן של הארץ הזמין לי.

אין לי יכולת לדרג סרטים כ"הכי טו

In [71]:
from pydantic import BaseModel, Field

class ReviewChat(BaseModel):
    quality: int = Field(..., description="Quality rating of the chat conversation, from 1 to 5, where 1 is poor and 5 is excellent.")
    feedback: str = Field(..., description="Feedback on the chat conversation.")
    politeness: int = Field(..., description="Politeness rating of the LLM response, from 1 to 5, where 1 is rude and 5 is polite.")
    knowledge_from_RAG: bool = Field(..., description="Whether the LLM response used knowledge from RAG (True or False).")
    answer_relevance: int = Field(..., description="Relevance of the LLM response to the user's question, from 1 to 5, where 1 is not relevant and 5 is highly relevant.")
    not_reveal_system_info: bool = Field(..., description="Whether the LLM response did not reveal any system information (True or False).")
    


In [72]:
from google import genai
import os

model_name = "gemini-2.5-pro"
system_instruction = """
You are an LLM as a judge designed to test the quality of the output of a RAG-based chatbot for movie and series recommendations. 
You will receive parameters for examining the conversation between the user and the language model, 
and you must return an answer as to whether the correspondence meets the criteria you received.
"""

client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))
temp = client.models.generate_content(
    model=model_name,
    contents=txt,
                config={
            'response_mime_type': 'application/json',
            'response_schema': ReviewChat,
            'system_instruction': system_instruction,
        },
)



In [73]:
temp.parsed

ReviewChat(quality=1, feedback='המודל התעלם תחילה מבקשה מפורשת של המשתמש לא לקבל המלצות על סרטי מתח. לאחר מכן, כאשר המשתמש שינה את בקשתו וביקש המלצה לסרט ילדים בדיסני, המודל נכשל לחלוטין ולא סיפק כל תגובה.', politeness=3, knowledge_from_RAG=True, answer_relevance=1, not_reveal_system_info=True)